In [15]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report

In [16]:
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data"
columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
df = pd.read_csv(url, names=columns)
df

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [17]:
df_encoded = pd.get_dummies(df, columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])

X = df_encoded.drop('class', axis=1)
y = df_encoded['class']

encoder = LabelEncoder()
y_trans = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_trans, test_size=0.2, random_state=42)

### XGBoost classifier

In [18]:
xgb_classifier = XGBClassifier(objective='multi:softmax')
xgb_classifier.fit(X_train, y_train)
y_pred_xgb = xgb_classifier.predict(X_test)

print("Classification Report: ")
print(classification_report(y_test, y_pred_xgb))

Classification Report: 
              precision    recall  f1-score   support

           0       1.00      0.92      0.96        83
           1       0.61      1.00      0.76        11
           2       1.00      1.00      1.00       235
           3       1.00      1.00      1.00        17

    accuracy                           0.98       346
   macro avg       0.90      0.98      0.93       346
weighted avg       0.99      0.98      0.98       346



### CatBoost Classifier

In [19]:
cat_classifier = CatBoostClassifier(iterations=150, depth=5, learning_rate=0.3, loss_function='MultiClass', verbose=15)
cat_classifier.fit(X_train, y_train)
y_pred_cat = cat_classifier.predict(X_test)

print("Classification Report: ")
print(classification_report(y_test, y_pred_cat))

0:	learn: 0.9474975	total: 1.75ms	remaining: 261ms
15:	learn: 0.2026547	total: 24.6ms	remaining: 206ms
30:	learn: 0.1136741	total: 46.9ms	remaining: 180ms
45:	learn: 0.0760837	total: 69ms	remaining: 156ms
60:	learn: 0.0548474	total: 90.8ms	remaining: 132ms
75:	learn: 0.0425822	total: 113ms	remaining: 110ms
90:	learn: 0.0346446	total: 134ms	remaining: 86.7ms
105:	learn: 0.0287692	total: 156ms	remaining: 64.7ms
120:	learn: 0.0247338	total: 177ms	remaining: 42.5ms
135:	learn: 0.0219709	total: 199ms	remaining: 20.5ms
149:	learn: 0.0195640	total: 220ms	remaining: 0us
Classification Report: 
              precision    recall  f1-score   support

           0       1.00      0.92      0.96        83
           1       0.59      0.91      0.71        11
           2       1.00      1.00      1.00       235
           3       0.94      1.00      0.97        17

    accuracy                           0.98       346
   macro avg       0.88      0.96      0.91       346
weighted avg       0.98    

In [20]:
accuracy_xgb = (y_test == y_pred_xgb).mean()
accuracy_cat = (y_test == y_pred_cat).mean()

print(f"Accuracy")
print(f"XGBoost: {accuracy_xgb}")
print(f"CatBoost: {accuracy_cat}")

Accuracy
XGBoost: 0.9797687861271677
CatBoost: 0.518109525877911
